In [1]:
import pandas as pd
!pip install dash plotly pandas
# Load your data
df = pd.read_csv('Prediction_file.csv')  # Replace with your actual file path
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import dash_table
df.columns

C:\Users\huzai\AppData\Local\Temp\ipykernel_12012\3863576163.py:9: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


Index(['Store', 'DayOfWeek', 'Date', 'Day', 'Month', 'Year', 'Sales',
       'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'WeekOfYear',
       'Month_name', 'Day_Count', 'CompetitionOpen', 'Promo2Open',
       'IsPromo2Month'],
      dtype='object')

In [2]:
import dash
from dash import dcc, html, Input, Output
import dash_table
import pandas as pd
import plotly.graph_objects as go

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sales Dashboard"),
    html.Div([
        html.Label("Enter Store ID (max 1115):"),
        dcc.Input(id='store-id', type='number', value=None, min=1, max=1115, step=1, debounce=True),
    ]),
    html.Div([
        dcc.Graph(id='sales-chart', className='six columns'),
        dcc.Graph(id='monthly-sales-chart', className='six columns')
    ], className='row'),
    html.Div([
        dcc.Graph(id='daywise-sales-chart', className='twelve columns')
    ], className='row'),
    html.Div([
        dcc.Graph(id='dayofweek-sales-chart', className='twelve columns')
    ], className='row'),
    html.Div([
        dcc.Graph(id='average-sales-comparison-chart', className='twelve columns')
    ], className='row'),
    html.Div([
        html.H2("Datewise Sales"),
        dash_table.DataTable(id='sales-table',
                             columns=[{"name": "Date", "id": "Date"}, {"name": "Sales", "id": "Sales"}],
                             style_table={'overflowX': 'scroll'},
                             style_cell={'minWidth': '100px', 'width': '150px', 'maxWidth': '300px', 'textAlign': 'center'})
    ], style={'padding': '20px'})
])
@app.callback(
    [Output('sales-chart', 'figure'),
     Output('monthly-sales-chart', 'figure'),
     Output('daywise-sales-chart', 'figure'),
     Output('dayofweek-sales-chart', 'figure'),
     Output('average-sales-comparison-chart', 'figure'),
     Output('sales-table', 'data')],
    [Input('store-id', 'value')]
)
def update_charts(store_id):
    if store_id is None:
        raise dash.exceptions.PreventUpdate

    # Assuming df is your DataFrame containing all sales data
    # Filter data for the selected store ID
    store_data = df[df['Store'] == store_id].copy()  # Make a copy to avoid modifying original data

    # Create a line chart of daily sales
    fig_sales = go.Figure()
    fig_sales.add_trace(go.Scatter(x=store_data['Date'], y=store_data['Sales'], mode='lines', name='Daily Sales'))
    fig_sales.update_layout(title={'text': f"Daily Sales for Store ID {store_id}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                            xaxis_title='Date',
                            yaxis_title='Sales')

    # Create a bar chart of monthly sales
    store_data['Date'] = pd.to_datetime(store_data['Date']).dt.date  # Extract only date part
    store_data['Month'] = pd.to_datetime(store_data['Date']).dt.to_period('M')
    monthly_sales = store_data.groupby('Month')['Sales'].sum().reset_index()

    fig_monthly = go.Figure()
    fig_monthly.add_trace(go.Bar(x=monthly_sales['Month'].astype(str), y=monthly_sales['Sales'], name='Monthly Sales',
                                 marker_color=['cornflowerblue', 'green']))
    fig_monthly.update_layout(title={'text': f"Monthly Sales for Store ID {store_id}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                              xaxis_title='Month',
                              yaxis_title='Sales',
                              barmode='group')  # Display bars in groups

    # Create a bar chart of day-wise sales
    daily_sales = store_data.groupby('Date')['Sales'].sum().reset_index()

    fig_daywise = go.Figure()
    fig_daywise.add_trace(go.Bar(x=daily_sales['Date'], y=daily_sales['Sales'], name='Day-wise Sales',
                                 marker_color='purple'))
    fig_daywise.update_layout(title={'text': f"Day-wise Sales for Store ID {store_id}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                              xaxis_title='Date',
                              yaxis_title='Sales')

    # Create a bar chart of day-of-week sales
    store_data['DayOfWeek'] = pd.to_datetime(store_data['Date']).dt.day_name()
    dayofweek_sales = store_data.groupby('DayOfWeek')['Sales'].sum().reset_index()

    fig_dayofweek = go.Figure()
    fig_dayofweek.add_trace(go.Bar(x=dayofweek_sales['DayOfWeek'], y=dayofweek_sales['Sales'], name='Day-of-Week Sales',
                                   marker_color='green'))
    fig_dayofweek.update_layout(title={'text': f"Day-of-Week Sales for Store ID {store_id}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                                xaxis_title='Day of Week',
                                yaxis_title='Sales')

    # Calculate the daily average sales of all other stores
    other_stores_data = df[df['Store'] != store_id]
    average_sales_other_stores = other_stores_data.groupby('Date')['Sales'].mean().reset_index()
    average_sales_store = store_data.groupby('Date')['Sales'].mean().reset_index()

    # Create a line chart to compare daily average sales
    fig_average_sales_comparison = go.Figure()
    fig_average_sales_comparison.add_trace(go.Scatter(x=average_sales_store['Date'], y=average_sales_store['Sales'],
                                                      mode='lines', name=f'Store {store_id} Daily Sales'))
    fig_average_sales_comparison.add_trace(go.Scatter(x=average_sales_other_stores['Date'], y=average_sales_other_stores['Sales'],
                                                      mode='lines', name='Other Stores Daily Average Sales', line=dict(dash='dash')))
    fig_average_sales_comparison.update_layout(
    title='Daily Average Sales Comparison',
    xaxis_title='Date',
    yaxis_title='Average Sales',
    legend=dict(font=dict(size=10), orientation='h', yanchor='top', y=0.95, xanchor='right', x=0.98),
    margin=dict(l=40, r=40, t=80, b=40),
    height=600,
    width=1000
)


    # Prepare data for the sales table, converting date to string for display
    sales_table_data = store_data[['Date', 'Sales']].copy()
    sales_table_data['Date'] = sales_table_data['Date'].astype(str)

    return fig_sales, fig_monthly, fig_daywise, fig_dayofweek, fig_average_sales_comparison, sales_table_data.to_dict('records')
if __name__ == '__main__':
    app.run_server(port=8050, debug=True)